# Part 2 (AI Course recommender)

In [6]:
! pip install fitz frontend

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [7]:
# Step 1: Extract skills from user CV (PDF or text)
import fitz  # PyMuPDF
import re
import openai

openai.api_key = "your-openai-api-key"

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def extract_skills_from_text(text):
    prompt = f"""
    Extract both technical and soft skills from this resume content:

    {text}

    Respond with a JSON:
    {{
        "technical_skills": [...],
        "soft_skills": [...]
    }}
    """
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that extracts skills from resumes."},
            {"role": "user", "content": prompt}
        ]
    )
    return response["choices"][0]["message"]["content"]

# Usage:
# cv_text = extract_text_from_pdf("path_to_user_cv.pdf")
# user_skills = extract_skills_from_text(cv_text)
# print(user_skills)


# Step 2: Extract job skills from a job description (scraped or pasted text)
def extract_skills_from_job_description(text):
    prompt = f"""
    Extract technical and soft skills required for the following job description:

    {text}

    Respond with a JSON:
    {{
        "technical_skills": [...],
        "soft_skills": [...]
    }}
    """
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You extract skills from job postings."},
            {"role": "user", "content": prompt}
        ]
    )
    return response["choices"][0]["message"]["content"]

# Usage:
# job_text = "Paste job description text here"
# job_skills = extract_skills_from_job_description(job_text)
# print(job_skills)


# Step 3: Compare skills
import json

def compare_skills(user_json, job_json):
    user = json.loads(user_json)
    job = json.loads(job_json)

    missing_tech = list(set(job["technical_skills"]) - set(user["technical_skills"]))
    missing_soft = list(set(job["soft_skills"]) - set(user["soft_skills"]))

    return {
        "missing_technical_skills": missing_tech,
        "missing_soft_skills": missing_soft
    }

RuntimeError: Directory 'static/' does not exist